## Install required libraries

In [1]:
!pip install obonet networkx
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install biopython

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.2 MB/s eta 0:00:0000:0100:01


## Import required modules

In [2]:
import numpy as np
import pandas as pd
import os
import duckdb as dd
import polars as pl
import h5py
import networkx as nx
import obonet as ob

import seaborn as sns
import matplotlib.pyplot as plt

from Bio import SeqIO
from typing import List

## Separating out the GO terms based on Ontology roots

In [3]:
def separate_go_terms_by_namespace(graph):
    """
    Separates GO terms from a networkx graph into BP, CC, and MF categories.

    Args:
        graph: A networkx MultiDiGraph loaded with obonet.

    Returns:
        A dictionary containing lists of GO term IDs for each namespace.
    """
    bp_terms = []
    cc_terms = []
    mf_terms = []

    # Iterate through every node in the graph, accessing its ID (node_id) and attributes (data)
    for node_id, data in graph.nodes(data=True):
        # The namespace is stored in the 'namespace' key of the node's data dictionary
        namespace = data.get('namespace')

        if namespace == 'biological_process':
            bp_terms.append(node_id)
        elif namespace == 'cellular_component':
            cc_terms.append(node_id)
        elif namespace == 'molecular_function':
            mf_terms.append(node_id)

    return {
        'BP': bp_terms,
        'CC': cc_terms,
        'MF': mf_terms
    }

## Load the Gene Ontology (GO) basic OBO file from the OBO Library

In [4]:
go_file_path = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
print(f"Loading Gene Ontology from {go_file_path}...")
go_graph = ob.read_obo(go_file_path)
print(f"Total terms loaded: {len(go_graph)}")

Loading Gene Ontology from /kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo...
Total terms loaded: 40122


## Separate the terms

In [5]:
separated_terms = separate_go_terms_by_namespace(go_graph)

print("\nGO Term Counts by Namespace:")
print(f"Biological Process (BP): {len(separated_terms['BP'])} terms")
print(f"Cellular Component (CC): {len(separated_terms['CC'])} terms")
print(f"Molecular Function (MF): {len(separated_terms['MF'])} terms")

print("\nFirst 5 Cellular Component (CC) terms:")
for term_id in separated_terms['CC'][:5]:
    # You can get the human-readable name using the graph data dictionary
    name = go_graph.nodes[term_id].get('name')
    print(f"- {term_id}: {name}")


GO Term Counts by Namespace:
Biological Process (BP): 25950 terms
Cellular Component (CC): 4041 terms
Molecular Function (MF): 10131 terms

First 5 Cellular Component (CC) terms:
- GO:0000015: phosphopyruvate hydratase complex
- GO:0000109: nucleotide-excision repair complex
- GO:0000110: nucleotide-excision repair factor 1 complex
- GO:0000111: nucleotide-excision repair factor 2 complex
- GO:0000112: nucleotide-excision repair factor 3 complex


## Roots of the three ontologies

In [ ]:
go_graph_edges = list(go_graph.edges)

#print(go_graph_edges[0])
print(go_graph.nodes["GO:0008150"])
print("*************")
print(go_graph.nodes["GO:0005575"])
print("*************")
print(go_graph.nodes["GO:0003674"])

In [ ]:
depths_from_bp_root = nx.shortest_path_length(go_graph, target="GO:0008150")
depths_from_cc_root = nx.shortest_path_length(go_graph, target="GO:0005575")
depths_from_mf_root = nx.shortest_path_length(go_graph, target="GO:0003674")

print("Max depth of BP ontology -> ",max(depths_from_bp_root.values()))
print("Max depth of CC ontology -> ",max(depths_from_cc_root.values()))
print("Max depth of MF ontology -> ",max(depths_from_mf_root.values()))

In [ ]:
cc_terms_at_depths = {}

for i in set(depths_from_cc_root.values()):
    cc_terms_at_depths[i] = [key for key, value in depths_from_cc_root.items() if value == i]

bp_terms_at_depths = {}

for i in set(depths_from_bp_root.values()):
    bp_terms_at_depths[i] = [key for key, value in depths_from_bp_root.items() if value == i]

mf_terms_at_depths = {}

for i in set(depths_from_mf_root.values()):
    mf_terms_at_depths[i] = [key for key, value in depths_from_mf_root.items() if value == i]

### Look at the number of terms at these depths

In [ ]:
for k,v in cc_terms_at_depths.items():
    print("{0} terms at depth {1}".format(len(cc_terms_at_depths[k]),k))

### Concatenate the CC terms at depths 0,1,7,8 and 9 since they are very less individually

In [ ]:
cc_depths_to_concatenate = [0,1,7,8,9]
concatenated_cc_terms = []

for d in cc_depths_to_concatenate:
    concatenated_cc_terms = concatenated_cc_terms + cc_terms_at_depths[d]

print(len(concatenated_cc_terms))

In [ ]:
for k,v in mf_terms_at_depths.items():
    print("{0} terms at depth {1}".format(len(mf_terms_at_depths[k]),k))

### Concatenate the MF terms at depths 0,1,8,9 and 10 since they are very less individually

In [ ]:
mf_depths_to_concatenate = [0,1,8,9,10]
concatenated_mf_terms = []

for d in mf_depths_to_concatenate:
    concatenated_mf_terms = concatenated_mf_terms + mf_terms_at_depths[d]

print(len(concatenated_mf_terms))

In [ ]:
for k,v in bp_terms_at_depths.items():
    print("{0} terms at depth {1}".format(len(bp_terms_at_depths[k]),k))

### Concatenate the terms at depths 0,1,9,10 and 11 since they are very less individually

In [ ]:
bp_depths_to_concatenate = [0,1,9,10,11]
concatenated_bp_terms = []

for d in bp_depths_to_concatenate:
    concatenated_bp_terms = concatenated_bp_terms + bp_terms_at_depths[d]

print(len(concatenated_bp_terms))

## Introduce the ESM2 embeddings of dimension 640

In [6]:
emb_df = pl.read_parquet('/kaggle/input/cafa6-protein-go-terms-feat-labels/train_protein_features_esm2_640.parquet')
emb_df.shape

(82404, 2)

In [7]:
emb_df.head(5)

protein_accession_id,embedding_arrays
str,"array[f32, 640]"
"""Q6ZUT6""","[-0.162842, 0.05481, … -0.221191]"
"""Q9JL26""","[-0.080872, -0.001384, … -0.09137]"
"""Q54H39""","[-0.041412, -0.049805, … -0.005016]"
"""O64682""","[-0.126343, -0.007771, … 0.010826]"
"""P26686""","[-0.085266, -0.078796, … 0.015625]"


## Bring in the training data; proteins and the corresponding GO terms

In [8]:
train_terms_df = pl.read_csv('/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv', separator='\t')
train_terms_df.shape

(537027, 3)

In [9]:
dd.sql("select count(distinct(EntryID)) as uniq_prots, count(distinct(term)) as uniq_terms from train_terms_df").pl()

uniq_prots,uniq_terms
i64,i64
82404,26125


In [10]:
train_terms_df.head(4)

EntryID,term,aspect
str,str,str
"""Q5W0B1""","""GO:0000785""","""C"""
"""Q5W0B1""","""GO:0004842""","""F"""
"""Q5W0B1""","""GO:0051865""","""P"""
"""Q5W0B1""","""GO:0006275""","""P"""


## Introduce the embeddings into the training data

In [11]:
train_terms_w_embeds = dd.sql("select distinct t1.protein_accession_id, t2.term, t2.aspect \
, t1.embedding_arrays as protein_embedding \
from emb_df t1 \
join train_terms_df t2 \
on t1.protein_accession_id = t2.EntryID").pl()

print(train_terms_w_embeds.shape)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(537027, 4)


In [12]:
dd.sql("select count(distinct(protein_accession_id)) as uniq_protein_accession_ids, \
count(distinct(term)) as uniq_go_terms from train_terms_w_embeds").pl()

uniq_protein_accession_ids,uniq_go_terms
i64,i64
82404,26125


## Create separate dataframes for each root ontology based on their depths

In [13]:
pl.Config(fmt_str_lengths=1000)
pl.Config.set_tbl_rows(1000)

polars.config.Config

### CC

In [ ]:
train_terms_w_embeds_cc_concat = train_terms_w_embeds.filter(pl.col("term").is_in(concatenated_cc_terms))
print("train_terms_w_embeds_cc_concat shape -> ",train_terms_w_embeds_cc_concat.shape)

train_terms_w_embeds_cc_2 = train_terms_w_embeds.filter(pl.col("term").is_in(cc_terms_at_depths[2]))
print("train_terms_w_embeds_cc_2 shape -> ",train_terms_w_embeds_cc_2.shape)

train_terms_w_embeds_cc_3 = train_terms_w_embeds.filter(pl.col("term").is_in(cc_terms_at_depths[3]))
print("train_terms_w_embeds_cc_3 shape -> ",train_terms_w_embeds_cc_3.shape)

train_terms_w_embeds_cc_4 = train_terms_w_embeds.filter(pl.col("term").is_in(cc_terms_at_depths[4]))
print("train_terms_w_embeds_cc_4 shape -> ",train_terms_w_embeds_cc_4.shape)

train_terms_w_embeds_cc_5 = train_terms_w_embeds.filter(pl.col("term").is_in(cc_terms_at_depths[5]))
print("train_terms_w_embeds_cc_5 shape -> ",train_terms_w_embeds_cc_5.shape)

train_terms_w_embeds_cc_6 = train_terms_w_embeds.filter(pl.col("term").is_in(cc_terms_at_depths[6]))
print("train_terms_w_embeds_cc_6 shape -> ",train_terms_w_embeds_cc_6.shape)

In [ ]:
print(dd.sql("select term, \
count(distinct(protein_accession_id)) as uniq_records from train_terms_w_embeds_cc_concat group by term \
order by count(distinct(protein_accession_id)) desc").pl())

## MF

In [ ]:
train_terms_w_embeds_mf_concat = train_terms_w_embeds.filter(pl.col("term").is_in(concatenated_mf_terms)) # 0,1,8,9,10
print("train_terms_w_embeds_mf_concat shape -> ",train_terms_w_embeds_mf_concat.shape)

train_terms_w_embeds_mf_2 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[2]))
print("train_terms_w_embeds_mf_2 shape -> ",train_terms_w_embeds_mf_2.shape)

train_terms_w_embeds_mf_3 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[3]))
print("train_terms_w_embeds_mf_3 shape -> ",train_terms_w_embeds_mf_3.shape)

train_terms_w_embeds_mf_4 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[4]))
print("train_terms_w_embeds_mf_4 shape -> ",train_terms_w_embeds_mf_4.shape)

train_terms_w_embeds_mf_5 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[5]))
print("train_terms_w_embeds_mf_5 shape -> ",train_terms_w_embeds_mf_5.shape)

train_terms_w_embeds_mf_6 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[6]))
print("train_terms_w_embeds_mf_6 shape -> ",train_terms_w_embeds_mf_6.shape)

train_terms_w_embeds_mf_7 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[7]))
print("train_terms_w_embeds_mf_7 shape -> ",train_terms_w_embeds_mf_7.shape)

## BP

In [ ]:
train_terms_w_embeds_bp_concat = train_terms_w_embeds.filter(pl.col("term").is_in(concatenated_bp_terms)) # 0,1,9,10,11
print("train_terms_w_embeds_bp_concat shape -> ",train_terms_w_embeds_bp_concat.shape)

train_terms_w_embeds_bp_2 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[2]))
print("train_terms_w_embeds_bp_2 shape -> ",train_terms_w_embeds_bp_2.shape)

train_terms_w_embeds_bp_3 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[3]))
print("train_terms_w_embeds_bp_3 shape -> ",train_terms_w_embeds_bp_3.shape)

train_terms_w_embeds_bp_4 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[4]))
print("train_terms_w_embeds_bp_4 shape -> ",train_terms_w_embeds_bp_4.shape)

train_terms_w_embeds_bp_5 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[5]))
print("train_terms_w_embeds_bp_5 shape -> ",train_terms_w_embeds_bp_5.shape)

train_terms_w_embeds_bp_6 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[6]))
print("train_terms_w_embeds_bp_6 shape -> ",train_terms_w_embeds_bp_6.shape)

train_terms_w_embeds_bp_7 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[7]))
print("train_terms_w_embeds_bp_7 shape -> ",train_terms_w_embeds_bp_7.shape)

train_terms_w_embeds_bp_8 = train_terms_w_embeds.filter(pl.col("term").is_in(mf_terms_at_depths[8]))
print("train_terms_w_embeds_bp_8 shape -> ",train_terms_w_embeds_bp_8.shape)

## These are for NN model training

In [14]:
train_terms_w_embeds_bp = train_terms_w_embeds.filter(pl.col("term").is_in(separated_terms['BP']))
print("train_terms_w_embeds_bp shape -> ",train_terms_w_embeds_bp.shape)
print(dd.sql("select count(distinct(term)) as terms \
, count(distinct(protein_accession_id)) as proteins \
from train_terms_w_embeds_bp").pl())

train_terms_w_embeds_cc = train_terms_w_embeds.filter(pl.col("term").is_in(separated_terms['CC']))
print("train_terms_w_embeds_cc shape -> ",train_terms_w_embeds_cc.shape)
print(dd.sql("select count(distinct(term)) as terms \
, count(distinct(protein_accession_id)) as proteins \
from train_terms_w_embeds_cc").pl())

train_terms_w_embeds_mf = train_terms_w_embeds.filter(pl.col("term").is_in(separated_terms['MF']))
print("train_terms_w_embeds_mf shape -> ",train_terms_w_embeds_mf.shape)
print(dd.sql("select count(distinct(term)) as terms \
, count(distinct(protein_accession_id)) as proteins \
from train_terms_w_embeds_mf").pl())

train_terms_w_embeds_bp shape ->  (250805, 4)
shape: (1, 2)
┌───────┬──────────┐
│ terms ┆ proteins │
│ ---   ┆ ---      │
│ i64   ┆ i64      │
╞═══════╪══════════╡
│ 16858 ┆ 59958    │
└───────┴──────────┘
train_terms_w_embeds_cc shape ->  (157770, 4)
shape: (1, 2)
┌───────┬──────────┐
│ terms ┆ proteins │
│ ---   ┆ ---      │
│ i64   ┆ i64      │
╞═══════╪══════════╡
│ 2651  ┆ 60292    │
└───────┴──────────┘
train_terms_w_embeds_mf shape ->  (128452, 4)
shape: (1, 2)
┌───────┬──────────┐
│ terms ┆ proteins │
│ ---   ┆ ---      │
│ i64   ┆ i64      │
╞═══════╪══════════╡
│ 6616  ┆ 58001    │
└───────┴──────────┘


In [ ]:
dd.sql("select term, count(distinct(protein_accession_id)) as proteins from train_terms_w_embeds_cc \
group by term having count(distinct(protein_accession_id)) >= 40 order by 2 desc limit 1500").pl()

## Get the most frequently occuring GO terms

In [ ]:
plot_df = dd.sql("select term, count(1) as freq from train_terms_w_embeds_mf_concat group by term \
having count(1) >= 40 order by count(1) desc ").pl()
plot_df_pandas = plot_df.to_pandas()

figure, axis = plt.subplots(1, 1, figsize=(12, 6))

bp = sns.barplot(ax=axis, x=plot_df_pandas.term, y=plot_df_pandas.freq)
bp.set_xticklabels(bp.get_xticklabels(), rotation=90, size = 6)
axis.set_title('Top frequent GO term IDs')
bp.set_xlabel("GO term IDs", fontsize = 12)
bp.set_ylabel("Count", fontsize = 12)
plt.show()

In [15]:
num_of_labels = 1500

def train_labels_per_ontology_pl(input_df: pl.DataFrame, input_num_of_labels: int) -> pl.DataFrame:
    """
    Filters a Polars DataFrame to include only the top N most frequent terms.
    
    Args:
        input_df: The input Polars DataFrame with a 'term' column.
        input_num_of_labels: The number of top terms to keep.
        
    Returns:
        A new Polars DataFrame containing only the top N frequent labels.
    """

    # 1. Calculate the top N frequent terms using Polars methods
    top_labels_df = input_df.group_by("term").count().sort("count", descending=True).limit(input_num_of_labels)
    #top_labels_df = input_df.group_by("term").agg(pl.col("protein_accession_id").n_unique().alias("unique_protein_count"))\
    #.filter(pl.col("unique_protein_count") >= input_num_of_labels)
    
    # 2. Extract these top terms into a Python list
    labels_list: List[str] = top_labels_df["term"].to_list()
    
    # 3. Filter the original DataFrame to keep only those terms
    train_labels = input_df.filter(pl.col("term").is_in(labels_list))
    
    print(f"Shape of filtered training data: {train_labels.shape}")
    print("Head of filtered training data:")
    print(train_labels.head())

    return train_labels


In [ ]:
result = (
    train_terms_w_embeds_cc_concat.group_by("term")
    .agg(pl.col("protein_accession_id").n_unique().alias("unique_protein_count"))
    .filter(pl.col("unique_protein_count") >= 50)
)

result["term"].to_list()

## For NN training

In [16]:
train_terms_cc = train_labels_per_ontology_pl(train_terms_w_embeds_cc, num_of_labels)
train_terms_mf = train_labels_per_ontology_pl(train_terms_w_embeds_mf, num_of_labels)
train_terms_bp = train_labels_per_ontology_pl(train_terms_w_embeds_bp, num_of_labels)

/tmp/ipykernel_38/2309332887.py:16: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  top_labels_df = input_df.group_by("term").count().sort("count", descending=True).limit(input_num_of_labels)


Shape of filtered training data: (154977, 4)
Head of filtered training data:
shape: (5, 4)
┌──────────────────────┬────────────┬────────┬─────────────────────────────────────┐
│ protein_accession_id ┆ term       ┆ aspect ┆ protein_embedding                   │
│ ---                  ┆ ---        ┆ ---    ┆ ---                                 │
│ str                  ┆ str        ┆ str    ┆ array[f32, 640]                     │
╞══════════════════════╪════════════╪════════╪═════════════════════════════════════╡
│ Q7RTN6               ┆ GO:0005829 ┆ C      ┆ [-0.105347, -0.012627, … -0.001822] │
│ Q6NSR3               ┆ GO:0005739 ┆ C      ┆ [-0.15918, 0.024796, … -0.10144]    │
│ Q91ZD4               ┆ GO:0060187 ┆ C      ┆ [-0.079102, -0.054108, … -0.019592] │
│ O75936               ┆ GO:0070062 ┆ C      ┆ [0.021271, -0.029724, … -0.052887]  │
│ Q06406               ┆ GO:0005634 ┆ C      ┆ [0.002611, -0.033356, … 0.040558]   │
└──────────────────────┴────────────┴────────┴─────────────

In [18]:
train_protein_labels_cc = train_terms_cc.select(['protein_accession_id','term'])
print(train_protein_labels_cc.head(5))

train_protein_labels_mf = train_terms_mf.select(['protein_accession_id','term'])
print(train_protein_labels_mf.head(5))

train_protein_labels_bp = train_terms_bp.select(['protein_accession_id','term'])
print(train_protein_labels_bp.head(5))

shape: (5, 2)
┌──────────────────────┬────────────┐
│ protein_accession_id ┆ term       │
│ ---                  ┆ ---        │
│ str                  ┆ str        │
╞══════════════════════╪════════════╡
│ Q7RTN6               ┆ GO:0005829 │
│ Q6NSR3               ┆ GO:0005739 │
│ Q91ZD4               ┆ GO:0060187 │
│ O75936               ┆ GO:0070062 │
│ Q06406               ┆ GO:0005634 │
└──────────────────────┴────────────┘
shape: (5, 2)
┌──────────────────────┬────────────┐
│ protein_accession_id ┆ term       │
│ ---                  ┆ ---        │
│ str                  ┆ str        │
╞══════════════════════╪════════════╡
│ Q14694               ┆ GO:0002039 │
│ Q9JMB7               ┆ GO:0005515 │
│ Q06406               ┆ GO:0003723 │
│ Q7XJK8               ┆ GO:0005515 │
│ P07451               ┆ GO:0005515 │
└──────────────────────┴────────────┘
shape: (5, 2)
┌──────────────────────┬────────────┐
│ protein_accession_id ┆ term       │
│ ---                  ┆ ---        │
│ str   

In [20]:
pivoted_df_cc = get_pivoted_df(train_protein_labels_cc)
print(pivoted_df_cc.shape)

pivoted_df_bp = get_pivoted_df(train_protein_labels_bp)
print(pivoted_df_bp.shape)

pivoted_df_mf = get_pivoted_df(train_protein_labels_mf)
print(pivoted_df_mf.shape)

shape: (5, 1_501)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ protein_a ┆ GO:000582 ┆ GO:000573 ┆ GO:006018 ┆ … ┆ GO:003258 ┆ GO:009741 ┆ GO:199043 ┆ GO:00442 │
│ ccession_ ┆ 9         ┆ 9         ┆ 7         ┆   ┆ 6         ┆ 3         ┆ 3         ┆ 98       │
│ id        ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ str       ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Q7RTN6    ┆ 1.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ Q6NSR3    ┆ 0.0       ┆ 1.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ Q91ZD4    ┆ 0.0       ┆ 0.0       ┆ 1.0       ┆ … ┆ 0.0       ┆ 0.0    

In [21]:
train_protein_features_cc = dd.sql("select distinct protein_accession_id, protein_embedding \
from train_terms_cc").pl()
print(train_protein_features_cc.head(5))

train_protein_features_bp = dd.sql("select distinct protein_accession_id, protein_embedding \
from train_terms_bp").pl()
print(train_protein_features_bp.head(5))

train_protein_features_mf = dd.sql("select distinct protein_accession_id, protein_embedding \
from train_terms_mf").pl()
print(train_protein_features_mf.head(5))

shape: (5, 2)
┌──────────────────────┬─────────────────────────────────────┐
│ protein_accession_id ┆ protein_embedding                   │
│ ---                  ┆ ---                                 │
│ str                  ┆ array[f32, 640]                     │
╞══════════════════════╪═════════════════════════════════════╡
│ Q09861               ┆ [-0.192383, 0.012848, … 0.013435]   │
│ P08F94               ┆ [-0.102478, -0.034332, … -0.038513] │
│ Q9H2U1               ┆ [-0.025726, -0.025574, … -0.053223] │
│ P9WH65               ┆ [-0.018997, -0.143066, … 0.091064]  │
│ Q6NT55               ┆ [-0.078186, -0.015106, … -0.0336]   │
└──────────────────────┴─────────────────────────────────────┘
shape: (5, 2)
┌──────────────────────┬────────────────────────────────────┐
│ protein_accession_id ┆ protein_embedding                  │
│ ---                  ┆ ---                                │
│ str                  ┆ array[f32, 640]                    │
╞══════════════════════╪═══════

In [22]:
pivoted_df_cc.write_parquet('train_protein_labels_cc.parquet')
train_protein_features_cc.write_parquet('train_protein_features_cc.parquet')

pivoted_df_mf.write_parquet('train_protein_labels_mf.parquet')
train_protein_features_mf.write_parquet('train_protein_features_mf.parquet')

pivoted_df_bp.write_parquet('train_protein_labels_bp.parquet')
train_protein_features_bp.write_parquet('train_protein_features_bp.parquet')

## For GPU LR training

## CC

In [ ]:
train_terms_cc_concat = train_labels_per_ontology_pl(train_terms_w_embeds_cc_concat, num_of_labels)
train_terms_cc_2 = train_labels_per_ontology_pl(train_terms_w_embeds_cc_2, num_of_labels)
train_terms_cc_3 = train_labels_per_ontology_pl(train_terms_w_embeds_cc_3, num_of_labels)
train_terms_cc_4 = train_labels_per_ontology_pl(train_terms_w_embeds_cc_4, num_of_labels)
train_terms_cc_5 = train_labels_per_ontology_pl(train_terms_w_embeds_cc_5, num_of_labels)
train_terms_cc_6 = train_labels_per_ontology_pl(train_terms_w_embeds_cc_6, num_of_labels)

## MF

In [ ]:
train_terms_mf_concat = train_labels_per_ontology_pl(train_terms_w_embeds_mf_concat, num_of_labels)
train_terms_mf_2 = train_labels_per_ontology_pl(train_terms_w_embeds_mf_2, num_of_labels)
train_terms_mf_3 = train_labels_per_ontology_pl(train_terms_w_embeds_mf_3, num_of_labels)
train_terms_mf_4 = train_labels_per_ontology_pl(train_terms_w_embeds_mf_4, num_of_labels)
train_terms_mf_5 = train_labels_per_ontology_pl(train_terms_w_embeds_mf_5, num_of_labels)
train_terms_mf_6 = train_labels_per_ontology_pl(train_terms_w_embeds_mf_6, num_of_labels)
train_terms_mf_7 = train_labels_per_ontology_pl(train_terms_w_embeds_mf_7, num_of_labels)

## BP

In [ ]:
train_terms_bp_concat = train_labels_per_ontology_pl(train_terms_w_embeds_bp_concat, num_of_labels)
train_terms_bp_2 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_2, num_of_labels)
train_terms_bp_3 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_3, num_of_labels)
train_terms_bp_4 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_4, num_of_labels)
train_terms_bp_5 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_5, num_of_labels)
train_terms_bp_6 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_6, num_of_labels)
train_terms_bp_7 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_7, num_of_labels)
train_terms_bp_8 = train_labels_per_ontology_pl(train_terms_w_embeds_bp_8, num_of_labels)

## CC

In [ ]:
train_protein_labels_cc_concat = train_terms_cc_concat.select(['protein_accession_id','term'])
print(train_protein_labels_cc_concat.head(5))

train_protein_labels_cc_2 = train_terms_cc_2.select(['protein_accession_id','term'])
print(train_protein_labels_cc_2.head(5))

train_protein_labels_cc_3 = train_terms_cc_3.select(['protein_accession_id','term'])
print(train_protein_labels_cc_3.head(5))

train_protein_labels_cc_4 = train_terms_cc_4.select(['protein_accession_id','term'])
print(train_protein_labels_cc_4.head(5))

train_protein_labels_cc_5 = train_terms_cc_5.select(['protein_accession_id','term'])
print(train_protein_labels_cc_5.head(5))

train_protein_labels_cc_6 = train_terms_cc_6.select(['protein_accession_id','term'])
print(train_protein_labels_cc_6.head(5))

## MF

In [ ]:
train_protein_labels_mf_concat = train_terms_mf_concat.select(['protein_accession_id','term'])
print(train_protein_labels_mf_concat.head(5))

train_protein_labels_mf_2 = train_terms_mf_2.select(['protein_accession_id','term'])
print(train_protein_labels_mf_2.head(5))

train_protein_labels_mf_3 = train_terms_mf_3.select(['protein_accession_id','term'])
print(train_protein_labels_mf_3.head(5))

train_protein_labels_mf_4 = train_terms_mf_4.select(['protein_accession_id','term'])
print(train_protein_labels_mf_4.head(5))

train_protein_labels_mf_5 = train_terms_mf_5.select(['protein_accession_id','term'])
print(train_protein_labels_mf_5.head(5))

train_protein_labels_mf_6 = train_terms_mf_6.select(['protein_accession_id','term'])
print(train_protein_labels_mf_6.head(5))

train_protein_labels_mf_7 = train_terms_mf_7.select(['protein_accession_id','term'])
print(train_protein_labels_mf_7.head(5))

## BP

In [ ]:
train_protein_labels_bp_concat = train_terms_bp_concat.select(['protein_accession_id','term'])
print(train_protein_labels_bp_concat.head(5))

train_protein_labels_bp_2 = train_terms_bp_2.select(['protein_accession_id','term'])
print(train_protein_labels_bp_2.head(5))

train_protein_labels_bp_3 = train_terms_bp_3.select(['protein_accession_id','term'])
print(train_protein_labels_bp_3.head(5))

train_protein_labels_bp_4 = train_terms_bp_4.select(['protein_accession_id','term'])
print(train_protein_labels_bp_4.head(5))

train_protein_labels_bp_5 = train_terms_bp_5.select(['protein_accession_id','term'])
print(train_protein_labels_bp_5.head(5))

train_protein_labels_bp_6 = train_terms_bp_6.select(['protein_accession_id','term'])
print(train_protein_labels_bp_6.head(5))

train_protein_labels_bp_7 = train_terms_bp_7.select(['protein_accession_id','term'])
print(train_protein_labels_bp_7.head(5))

train_protein_labels_bp_8 = train_terms_bp_8.select(['protein_accession_id','term'])
print(train_protein_labels_bp_8.shape)
print(train_protein_labels_bp_8.head(5))

In [19]:
def get_pivoted_df(input_df: pl.DataFrame) -> pl.DataFrame:
    train_protein_labels_w_term_presence = input_df.with_columns(
        pl.lit(1.0).alias("presence")
    )

    pivoted_df = train_protein_labels_w_term_presence.pivot(
        index="protein_accession_id",
        on="term",
        values="presence",  
        aggregate_function="sum"
    )
    
    pivoted_df = pivoted_df.fill_null(0)
    print(pivoted_df.head(5))
    return pivoted_df

## CC

In [ ]:
pivoted_df_cc_concat = get_pivoted_df(train_protein_labels_cc_concat)
pivoted_df_cc_2 = get_pivoted_df(train_protein_labels_cc_2)
pivoted_df_cc_3 = get_pivoted_df(train_protein_labels_cc_3)
pivoted_df_cc_4 = get_pivoted_df(train_protein_labels_cc_4)
pivoted_df_cc_5 = get_pivoted_df(train_protein_labels_cc_5)
pivoted_df_cc_6 = get_pivoted_df(train_protein_labels_cc_6)

In [ ]:
print(pivoted_df_cc_concat.shape)
print(pivoted_df_cc_2.shape)
print(pivoted_df_cc_3.shape)
print(pivoted_df_cc_4.shape)
print(pivoted_df_cc_5.shape)
print(pivoted_df_cc_6.shape)

## MF

In [ ]:
pivoted_df_mf_concat = get_pivoted_df(train_protein_labels_mf_concat)
pivoted_df_mf_2 = get_pivoted_df(train_protein_labels_mf_2)
pivoted_df_mf_3 = get_pivoted_df(train_protein_labels_mf_3)
pivoted_df_mf_4 = get_pivoted_df(train_protein_labels_mf_4)
pivoted_df_mf_5 = get_pivoted_df(train_protein_labels_mf_5)
pivoted_df_mf_6 = get_pivoted_df(train_protein_labels_mf_6)
pivoted_df_mf_7 = get_pivoted_df(train_protein_labels_mf_7)

In [ ]:
print(pivoted_df_mf_concat.shape)
print(pivoted_df_mf_2.shape)
print(pivoted_df_mf_3.shape)
print(pivoted_df_mf_4.shape)
print(pivoted_df_mf_5.shape)
print(pivoted_df_mf_6.shape)
print(pivoted_df_mf_7.shape)

## BP

In [ ]:
pivoted_df_bp_concat = get_pivoted_df(train_protein_labels_bp_concat)
pivoted_df_bp_2 = get_pivoted_df(train_protein_labels_bp_2)
pivoted_df_bp_3 = get_pivoted_df(train_protein_labels_bp_3)
pivoted_df_bp_4 = get_pivoted_df(train_protein_labels_bp_4)
pivoted_df_bp_5 = get_pivoted_df(train_protein_labels_bp_5)
pivoted_df_bp_6 = get_pivoted_df(train_protein_labels_bp_6)
pivoted_df_bp_7 = get_pivoted_df(train_protein_labels_bp_7)
pivoted_df_bp_8 = get_pivoted_df(train_protein_labels_bp_8)

In [ ]:
print(pivoted_df_bp_concat.shape)
print(pivoted_df_bp_2.shape)
print(pivoted_df_bp_3.shape)
print(pivoted_df_bp_4.shape)
print(pivoted_df_bp_5.shape)
print(pivoted_df_bp_6.shape)
print(pivoted_df_bp_7.shape)
print(pivoted_df_bp_8.shape)

## CC

In [ ]:
train_protein_features_cc_concat = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_cc_concat").pl()
print(train_protein_features_cc_concat.head(5))

train_protein_features_cc_2 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_cc_2").pl()
print(train_protein_features_cc_2.head(5))

train_protein_features_cc_3 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_cc_3").pl()
print(train_protein_features_cc_3.head(5))

train_protein_features_cc_4 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_cc_4").pl()
print(train_protein_features_cc_4.head(5))

train_protein_features_cc_5 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_cc_5").pl()
print(train_protein_features_cc_5.head(5))

train_protein_features_cc_6 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_cc_6").pl()
print(train_protein_features_cc_6.head(5))

## MF

In [ ]:
train_protein_features_mf_concat = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_concat").pl()
print(train_protein_features_mf_concat.head(5))

train_protein_features_mf_2 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_2").pl()
print(train_protein_features_mf_2.head(5))

train_protein_features_mf_3 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_3").pl()
print(train_protein_features_mf_3.head(5))

train_protein_features_mf_4 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_4").pl()
print(train_protein_features_mf_4.head(5))

train_protein_features_mf_5 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_5").pl()
print(train_protein_features_mf_5.head(5))

train_protein_features_mf_6 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_6").pl()
print(train_protein_features_mf_6.head(5))

train_protein_features_mf_7 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_mf_7").pl()
print(train_protein_features_mf_7.head(5))

## BP

In [ ]:
train_protein_features_bp_concat = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_concat").pl()
print(train_protein_features_bp_concat.head(5))

train_protein_features_bp_2 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_2").pl()
print(train_protein_features_bp_2.head(5))

train_protein_features_bp_3 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_3").pl()
print(train_protein_features_bp_3.head(5))

train_protein_features_bp_4 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_4").pl()
print(train_protein_features_bp_4.head(5))

train_protein_features_bp_5 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_5").pl()
print(train_protein_features_bp_5.head(5))

train_protein_features_bp_6 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_6").pl()
print(train_protein_features_bp_6.head(5))

train_protein_features_bp_7 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_7").pl()
print(train_protein_features_bp_7.head(5))

train_protein_features_bp_8 = dd.sql("select distinct protein_accession_id, protein_embedding from train_terms_bp_8").pl()
print(train_protein_features_bp_8.head(5))

## CC

In [ ]:
pivoted_df_cc_concat.write_parquet('train_protein_labels_cc_concat.parquet')
train_protein_features_cc_concat.write_parquet('train_protein_features_cc_concat.parquet')

pivoted_df_cc_2.write_parquet('train_protein_labels_cc_2.parquet')
train_protein_features_cc_2.write_parquet('train_protein_features_cc_2.parquet')

pivoted_df_cc_3.write_parquet('train_protein_labels_cc_3.parquet')
train_protein_features_cc_3.write_parquet('train_protein_features_cc_3.parquet')

pivoted_df_cc_4.write_parquet('train_protein_labels_cc_4.parquet')
train_protein_features_cc_4.write_parquet('train_protein_features_cc_4.parquet')

pivoted_df_cc_5.write_parquet('train_protein_labels_cc_5.parquet')
train_protein_features_cc_5.write_parquet('train_protein_features_cc_5.parquet')

pivoted_df_cc_6.write_parquet('train_protein_labels_cc_6.parquet')
train_protein_features_cc_6.write_parquet('train_protein_features_cc_6.parquet')

## MF

In [ ]:
pivoted_df_mf_concat.write_parquet('train_protein_labels_mf_concat.parquet')
train_protein_features_mf_concat.write_parquet('train_protein_features_mf_concat.parquet')

pivoted_df_mf_2.write_parquet('train_protein_labels_mf_2.parquet')
train_protein_features_mf_2.write_parquet('train_protein_features_mf_2.parquet')

pivoted_df_mf_3.write_parquet('train_protein_labels_mf_3.parquet')
train_protein_features_mf_3.write_parquet('train_protein_features_mf_3.parquet')

pivoted_df_mf_4.write_parquet('train_protein_labels_mf_4.parquet')
train_protein_features_mf_4.write_parquet('train_protein_features_mf_4.parquet')

pivoted_df_mf_5.write_parquet('train_protein_labels_mf_5.parquet')
train_protein_features_mf_5.write_parquet('train_protein_features_mf_5.parquet')

pivoted_df_mf_6.write_parquet('train_protein_labels_mf_6.parquet')
train_protein_features_mf_6.write_parquet('train_protein_features_mf_6.parquet')

pivoted_df_mf_7.write_parquet('train_protein_labels_mf_7.parquet')
train_protein_features_mf_7.write_parquet('train_protein_features_mf_7.parquet')

## BP

In [ ]:
pivoted_df_bp_concat.write_parquet('train_protein_labels_bp_concat.parquet')
train_protein_features_bp_concat.write_parquet('train_protein_features_bp_concat.parquet')

pivoted_df_bp_2.write_parquet('train_protein_labels_bp_2.parquet')
train_protein_features_bp_2.write_parquet('train_protein_features_bp_2.parquet')

pivoted_df_bp_3.write_parquet('train_protein_labels_bp_3.parquet')
train_protein_features_bp_3.write_parquet('train_protein_features_bp_3.parquet')

pivoted_df_bp_4.write_parquet('train_protein_labels_bp_4.parquet')
train_protein_features_bp_4.write_parquet('train_protein_features_bp_4.parquet')

pivoted_df_bp_5.write_parquet('train_protein_labels_bp_5.parquet')
train_protein_features_bp_5.write_parquet('train_protein_features_bp_5.parquet')

pivoted_df_bp_6.write_parquet('train_protein_labels_bp_6.parquet')
train_protein_features_bp_6.write_parquet('train_protein_features_bp_6.parquet')

pivoted_df_bp_7.write_parquet('train_protein_labels_bp_7.parquet')
train_protein_features_bp_7.write_parquet('train_protein_features_bp_7.parquet')

pivoted_df_bp_8.write_parquet('train_protein_labels_bp_8.parquet')
train_protein_features_bp_8.write_parquet('train_protein_features_bp_8.parquet')

In [ ]:
test_prot_ids = [seq_record.id 
                 for seq_record in 
                 SeqIO.parse("/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta", "fasta")]

In [ ]:
test_emb_df = pl.read_parquet('/kaggle/input/cafa6-protein-go-terms-feat-labels/test_protein_features_esm2_480.parquet')
test_emb_df.shape

In [ ]:
test_embeds = test_emb_df.filter(pl.col("protein_accession_id").is_in(test_prot_ids))
test_embeds.shape

In [ ]:
test_emb_df.head(4)

In [ ]:
dd.sql("select count(distinct(protein_accession_id)) as uniq_protein_accession_ids from test_embeds").pl()